# GIS2RHESSys



**Introduction:**

GIS2RHESSys is designed to setup RHESSysEastCoast model. There are two parts in this model setup process. First part is to setup GIS dataset from the raw data user provide. GIS processes in the first part may include create GRASS dataset, import/reproject/resample elevation data, delineate catchment, analyze drainage & flow accumulation, set climate zones, identify riparian boundary, classify soil textures, import and translate LULC data to RHESSys model, distribute forest community, and identify surface preferential flow path and transports. The guide in the first part is in a list format, listing examples of modules for certain GIS processes. This is not a JupyterNotebook for users to execute codes listed below. Users may copy/paste the codes to assemble a workflow bash/shell script running on local machine.

**Software / computer requirements for GIS2RHESSys:**
- Max OS / LINUX (e.g., Urbantu) 
- GRASS 7.x installed and its associated libraries (see GRASS GIS website for details)
- R 3.6.x or above
- R package “rgrass7” and its dependences

## Headers and paths for all scripts

0. **Set system paths to GRASS, R, and Github libraries**

<div class="alert alert-block alert-info">
The PATH settings below are for MAC OS. Users may need to modify some paths for the LINUX environment.
</div>

In [ ]:
# path setup below is for Mac OS
# adding R paths
export PATH=$PATH:/usr/local/bin
export PATH=$PATH:/Library/Frameworks/R.framework/Resources
# adding GRASS bin paths
grassApp=$(ls /Applications | grep GRASS)
grassVersion=$(ls /Applications | grep GRASS | awk -F'[.-]' '{print $2"."$3}')
grassCMD=$(ls /Applications/"$grassApp"/Contents/Resources/bin | grep grass)
export PATH=$PATH:/Applications/"$grassApp"/Contents/Resources/bin:/Applications/"$grassApp"/Contents/Resources/scripts:/etc:/usr/lib
# adding GRASS environment variables: GISBASE_USER and GISBASE_SYSTEM in Grass.sh
export PATH=$PATH:~/Library/GRASS/"$grassVersion"/Modules/bin:~/Library/GRASS/"$grassVersion"/Modules/scripts
# adding GDAL and EPSG code lookup paths (Running gdal-config --datadir shows where GDAL searches for gcs.csv)
export GDAL_DATA=/Applications/"$grassApp"/Contents/Resources/share/gdal
GITHUBLIBRARIES="https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/libraries"


In [ ]:
# path setup below is for UVA RIVANNA singularity rhessys_v3.img
grassCMD='grass'
GITHUBLIBRARIES="https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/libraries"


<div class="alert alert-block alert-info">
Enter the path to the project directory.<br>
When search for UTM EPSG code, we recommend to use datum NAD83 for the U.S. catchments.<br>
The spatial resolution (e.g., 3/10/30 m) of the elevation raster is often the spatial resolution of the RHESSys model.
</div>

In [ ]:
PROJDIR='.' # path to the project location;
EPSGCODE='EPSG:26918' # EPSG code for NAD83 UTM ##N for the catchment
RESOLUTION=10 #spatial resolution (meters) of the grids
RHESSysNAME='rhessys' # e.g., rhessys
GISDBASE="$PROJDIR"/grass_dataset
LOCATION_NAME="$RHESSysNAME"
LOCATION="$GISDBASE"/$LOCATION_NAME
MAPSET=PERMANENT

## Modules for GIS process

### 1.1 Initiate project directory and set up GRASS GIS dataset for RHESSys modeling  

<div class="alert alert-block alert-info">
Create project directory structures and RHESSys model directory structures. We only do this once initial although these commands do not overwrite existing directories.
</div>

In [ ]:
### ... create rhessys folder structures
mkdir "$PROJDIR"/"$RHESSysNAME"
mkdir "$PROJDIR"/"$RHESSysNAME"/defs
mkdir "$PROJDIR"/"$RHESSysNAME"/flows
mkdir "$PROJDIR"/"$RHESSysNAME"/worldfiles
mkdir "$PROJDIR"/"$RHESSysNAME"/clim
mkdir "$PROJDIR"/"$RHESSysNAME"/tecfiles
mkdir "$PROJDIR"/"$RHESSysNAME"/output

### 1.2 Initiate GRASS dataset

<div class="alert alert-block alert-info">
Create project GIS database. We only do this once initial although this command does not overwrite existing database.
</div>

In [ ]:
### ... create grass database
mkdir "$PROJDIR"/grass_dataset # does not overwrite
$grassCMD -c $EPSGCODE -e "$LOCATION" 


### 1.3 **Import and process of elevation data**

<div class="alert alert-block alert-info">
(Option 1) Assume that we need to change the elevation raster projection and resample into desired spatial resolution.<br> a) We ask GRASS to read in the elevation raster source and then resample the spatial scale<br> b) We export the sampled elevation information into a new raster "demXXm.tif", where XX is the spatial resoluted defined above<br> c) We import the "demXXm.tif" to the project GIS database with reprojection process (based on gdal) and set the project GIS database spatial extension and resolution to match the "demXXm.tif".
</div>

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec r.in.gdal -e --overwrite input="$PROJDIR"/raw_data/DEM1m.tif output=demRAW location=elevationRAW
LOCATIONDEM="$GISDBASE"/elevationRAW
$grassCMD "$LOCATIONDEM"/"$MAPSET" --exec g.region raster=demRAW
$grassCMD "$LOCATIONDEM"/"$MAPSET" --exec g.region res=$RESOLUTION -a -p
$grassCMD "$LOCATIONDEM"/"$MAPSET" --exec r.resamp.stats -w input=demRAW output=dem$RESOLUTION'm' ### skip this if no resampling spatial scale
$grassCMD "$LOCATIONDEM"/"$MAPSET" --exec r.out.gdal --overwrite input=dem$RESOLUTION'm' output="$PROJDIR"/raw_data/dem$RESOLUTION'm.tif' format=GTiff
$grassCMD "$LOCATION"/"$MAPSET" --exec r.in.gdal -o -e --overwrite input="$PROJDIR"/raw_data/dem$RESOLUTION'm.tif' output=dem
$grassCMD "$LOCATION"/"$MAPSET" --exec g.region raster=dem

<div class="alert alert-block alert-info">
(Option 2) If the source elevation raster has the correct projection (UTM) and resolution, then we can just simply import it to the project GIS dataset and set the project GIS database spatial extension and resolution to match the elevation raster.
</div>

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec r.in.gdal -o -e --overwrite input="$PROJDIR"/raw_data/dem$RESOLUTION'm.tif' output=dem
$grassCMD "$LOCATION"/"$MAPSET" --exec g.region raster=dem

### 1.4 **Define catchment outlet**

<div class="alert alert-block alert-info">
(Option 1) Use Lat/Long (WSG84 decimal degree) to define catchment outlet. Note that keep the negative sign if applied.
</div>

In [ ]:
gageLat='39.47947' # catchment outlet WSG84 Lat (decimal degree)
gageLong='-76.67803' # catchment outlet WSG84 Long (decimal degree; includes the negative sign if applied)
eval $($grassCMD "$LOCATION"/$MAPSET --exec m.proj -i coordinates=$gageLong,$gageLat separator=space | awk '{print "xyCoord=" $1 "," $2}')
echo $xyCoord | $grassCMD "$LOCATION"/"$MAPSET" --exec v.in.ascii in=- out=outlet x=1 y=2 separator=, --overwrite

<div class="alert alert-block alert-info">
(Option 2) Import a catchment outlet shapefile. Note that this shapefile should contain only ONE point.
</div>

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec v.in.ogr --overwrite input="$PROJDIR"/raw_data/USGS_Delight.shp output=outlet location=outletRAW
LOCATIONOUTLET="$GISDBASE"'/outletRAW'
$grassCMD "$LOCATION"/"$MAPSET" --exec v.proj --overwrite location=outletRAW mapset=PERMANENT input=outlet output=outlet
rm -rf "$LOCATIONOUTLET"

### 1.5 **Catchment delineation**

<div class="alert alert-block alert-info">
User needs to provide an estimation of catchment area (m^2) and threshold values to define modeling stream network. <br>
The threshold value is an area (m^2), e.g., 620000 m^2 = 62 ha, that defines the extension of a major stream channels for RHESSys; it also affects the hillslope and subcatchment structures. <br>
The second threshold value is also an area, e.g., 100000 m^2 = 10 ha, that defines the full extension of a stream network, including head water channels, for riparian delineation and surface water routing.
</div>

In [ ]:
expectedDrainageArea=10956000 # meter sq.
expectedThresholdModelStr=620000 # meter sq.
expectedThresholdStrExt=100000 # meter sq.

In [ ]:
GRASS_thres=$(($expectedThresholdModelStr/$RESOLUTION/$RESOLUTION)) # grid cell for stream network and hillslope configuration
GRASS_thresII=$(($expectedThresholdStrExt/$RESOLUTION/$RESOLUTION))
GRASS_drainarea_lowerbound=$((98*$expectedDrainageArea/$RESOLUTION/$RESOLUTION/100)) # (allow 2% error)
GRASS_drainarea_upperbound=$((102*$expectedDrainageArea/$RESOLUTION/$RESOLUTION/100)) # (allow 2% error)
#
curl -s "$GITHUBLIBRARIES"/grass_delineation_1.sh | $grassCMD "$LOCATION"/"$MAPSET" --exec bash -s $GRASS_thres $GRASS_drainarea_lowerbound $GRASS_drainarea_upperbound
curl -s "$GITHUBLIBRARIES"/basin_determine.R | $grassCMD "$LOCATION"/"$MAPSET" --exec R --slave
$grassCMD "$LOCATION"/"$MAPSET" --exec r.watershed elevation=dem threshold=$GRASS_thresII stream=strExt --overwrite # full stream extension;
curl -s "$GITHUBLIBRARIES"/grass_delineation_2.sh | $grassCMD "$LOCATION"/"$MAPSET" --exec bash -s

<div class="alert alert-block alert-info">
When this procedure is completed, a catchment is delineated, along with slope, aspect, wettness index (D-inf), east & west horizon sun angles, hillslope IDs, subcatchment IDs, patch IDs, streams, flow accumulation, and drainage direction (D-Inf). GIS computation region and mask are also set by the delineated catchment. 
</div>

### 1.6 **Define climate zones**

<div class="alert alert-block alert-info">
(Option 1) The coarse climate zone setting  
</div>

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="zone_hill = hill"

<div class="alert alert-block alert-info">
(Option 2) Using statistical cluster analysis, we delineate the elevation bands and further partition the bands by aspect, slope, and hillslopes. The resulting climate zone is finer than the coarse setting (option 1) but not as fine as the finest setting (option 3).
</div>

In [ ]:
curl -s "$GITHUBLIBRARIES"/zone_cluster.R | $grassCMD "$LOCATION"/"$MAPSET" --exec R --slave --args dem slope aspect hill zone_cluster

<div class="alert alert-block alert-info">
(Option 3) The finest climate zone setting, same as elevation resolution. This also increases the RHESSys computation time
</div>

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="zone_patch = patch"

### 1.7 **Define isohyet (MUST)**

<div class="alert alert-block alert-info">
(Option 1) import ioshyet raster
</div>

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec r.in.gdal -o --overwrite input="$PROJDIR"/raw_data/isohyet.tif output=isohyet

<div class="alert alert-block alert-info">
(Option 2) calculate isohyet by raster calculator (the calculation below is just an example)
</div>

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="isohyet = (1 +(4.268e-04)*(dem-1200) +(-6.799e-05)*slope +(5.845e-03)*sin(aspect*0.01745329)*sin(slope*0.01745329) +(-1.267e-02)*cos(aspect*0.01745329)*sin(slope*0.01745329))*1.45817781"

<div class="alert alert-block alert-info">
(Option 3) no ioshyet --> set ioshyet to one everywhere
</div>

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="isohyet = 1"

### 1.8 **Soil texture from SSURGO**

<div class="alert alert-block alert-info">
This step requires the SSURGO spatial and tabular dataset. <br>
Users need to set the path to the SSURGO dataset, e.g., "$PROJDIR"/raw_data/MD005 <br>
The procedure below reprojects the SSURGO mukey (areal polygon) shapefile onto the project GIS database, extract soil information from the tabular dataset, and map the extracted soil information to the corresponding mukey polygons.
</div>

<div class="alert alert-block alert-info">
(Option 1) Simple soil texture classification; see USDA soil classes for details
</div>

In [ ]:
downloadedSSURGOdirectoryPATH="$PROJDIR"/raw_data/MD005
$grassCMD "$LOCATION"/"$MAPSET" --exec v.in.ogr --overwrite input="$downloadedSSURGOdirectoryPATH"/spatial/soilmu_a_"$(echo ${downloadedSSURGOdirectoryPATH##*/} | tr '[A-Z]' '[a-z]')".shp output=ssurgo location=soilRAW
LOCATIONSOIL="$GISDBASE"/soilRAW
### ... import (reprojected) soil data to ""$LOCATION"/$MAPSET" and convert shape-polygon to raster soil (USDA) classes
$grassCMD "$LOCATION"/"$MAPSET" --exec v.proj --overwrite location=soilRAW mapset=PERMANENT input=ssurgo output=ssurgo
$grassCMD "$LOCATION"/"$MAPSET" --exec v.to.rast --overwrite input=ssurgo use=cat output=soil_ssurgo
$grassCMD "$LOCATION"/"$MAPSET" --exec v.out.ascii --overwrite input=ssurgo type=centroid output="$PROJDIR"/"$RHESSysNAME"/soil_cat_mukey.csv columns=MUKEY format=point separator=comma

In [ ]:
curl -s https://raw.githubusercontent.com/laurencelin/ssurgo_extraction/master/ssurgo_extraction.R | R --slave --args "$downloadedSSURGOdirectoryPATH"
curl -s https://raw.githubusercontent.com/laurencelin/ssurgo_extraction/master/ssurgo_soiltexture2gis.R | $grassCMD "$LOCATION"/"$MAPSET" --exec R --slave --args "$PROJDIR"/"$RHESSysNAME"/soil_cat_mukey.csv "$downloadedSSURGOdirectoryPATH"/soil_mukey_texture.csv
rm -rf "$LOCATIONSOIL"

<div class="alert alert-block alert-info">
If filling missing data or gaps is needed, below is a command for the job. Please read documentations of "r.fill.stats", a build-in GRASS function, to know how to use it correct.
</div>

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec r.fill.stats -k --overwrite input=soil_texture output=soil_texture_fill distance=30 mode=mode power=2.0

<div class="alert alert-block alert-info">
(Option 2) Explicit soil information for local catchment, exacted from SSURGO database
</div>

In [ ]:
downloadedSSURGOdirectoryPATH="$PROJDIR"/raw_data/MD005
$grassCMD "$LOCATION"/"$MAPSET" --exec v.in.ogr --overwrite input="$downloadedSSURGOdirectoryPATH"/spatial/soilmu_a_"$(echo ${downloadedSSURGOdirectoryPATH##*/} | tr '[A-Z]' '[a-z]')".shp output=ssurgo location=soilRAW
LOCATIONSOIL="$GISDBASE"/soilRAW
### ... import (reprojected) soil data to ""$LOCATION"/$MAPSET" and convert shape-polygon to raster soil (USDA) classes
$grassCMD "$LOCATION"/"$MAPSET" --exec v.proj --overwrite location=soilRAW mapset=PERMANENT input=ssurgo output=ssurgo
$grassCMD "$LOCATION"/"$MAPSET" --exec v.to.rast --overwrite input=ssurgo use=cat output=soil_ssurgo
$grassCMD "$LOCATION"/"$MAPSET" --exec v.out.ascii --overwrite input=ssurgo type=centroid output="$PROJDIR"/"$RHESSysNAME"/soil_cat_mukey.csv columns=MUKEY format=point separator=comma

In [ ]:
curl -s https://raw.githubusercontent.com/laurencelin/ssurgo_extraction/master/ssurgo_fullextraction_step1.R | R --slave --args "$downloadedSSURGOdirectoryPATH"
curl -s https://raw.githubusercontent.com/laurencelin/ssurgo_extraction/master/ssurgo_fullextraction_step2.R | R --slave --args "$downloadedSSURGOdirectoryPATH" "$PROJDIR"/"$RHESSysNAME"/soil_cat_mukey.csv "$PROJDIR"/"$RHESSysNAME"/soil_mukey_rhessys.csv
curl -s https://raw.githubusercontent.com/laurencelin/ssurgo_extraction/master/ssurgo_fullextraction_step3.R | $grassCMD "$LOCATION"/"$MAPSET" --exec R --slave --args soil_ssurgo "$PROJDIR"/"$RHESSysNAME"/soil_mukey_rhessys.csv
rm -rf "$LOCATIONSOIL"

<div class="alert alert-block alert-info">
If filling missing data or gaps is needed, below is a command for the job. Please read documentations of "r.fill.stats", a build-in GRASS function, to know how to use it correct.
</div>

### 1.9 **Processing Landuse & landcover data**

![title](https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/LULC.png)

<div class="alert alert-block alert-info">
An important concept: Landuse & landcover (LULC) data provide % coverages of pre-defined classes on each grid (30, 10, 5, 3, 1 meters). These pre-defined classes could be very general. For stances, deciduous forest class (code 41 in NLCD) does not specifically define which decidous species or their composition to form that forest, nor provide the forest density / LAI value. All we learned from the 30-m coded 41 grid is that it's largely covered by deciduous forest. This is the reason why we need to specifically define the forest for the model in the next step. Another example, urban class 23 provides two pieces of information: urbanized landscape and median density of urbanization. These information about urban are imprecise for the model. To describe urbanization to the model, we need to let the model know how much rooftop covered area, how much imprevious surfaces, how much paved road covered, how storm drainage connect to different parts of landscape, and how much urban canopy still remained within the urbanized area. This motivates us to use high-resolution LULC dataset (e.g., EPA EnviroAtlas and Chesapeake Conservancy) with great detailed and specific LULC classes. To make use of these great detailed LULC information at the modeling scale (3, 10, or 30 m), we need to aggregate and lump information into the following compositions for each modeling patch:<br>
- forest composition (forestFrac) <br>
- shrub composition (shurbFrac) <br>
- crop composition (cropFrac) <br>
- grass composition (lawnFrac) <br>
- imperviousness composition (impFrac) <br>
<br>
For imperviousness compositional fraction, we further partition this fraction into different catagories:<br>
- impervious structure roof (roofFrac) <br>
- impervious parking/non-structural surface (drivewayFrac) <br>
- impervious (paved) road (pavedroadFrac) <br>
<br>
</div>

<div class="alert alert-block alert-info">
To translate LULC classifications into the compositions above, we developed an approach to define each pre-defined LULC class in the dataset by the compositions. We illustrated that with an example below.
</div>

For example, NLCD code 42 - deciduous forest, 22 low-density urban, and 24 high-density urban,

![title](https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/lulcTable.png)

<div class="alert alert-block alert-info">
Above is the concept, and below lists different modules to accommodate different LULC sources
</div>

<div class="alert alert-block alert-info">
(Option 1) NLCD LULC classification for remotely forested landscape
</div>

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec r.in.gdal -e --overwrite input="$PROJDIR"/raw_data/'LULC1m.tif' output=lulcRAW location=lulcRAW
LOCATIONLULC="$GISDBASE"/lulcRAW
$grassCMD "$LOCATION"/"$MAPSET" --exec r.to.vect input=patch output=patch type=area
$grassCMD "$LOCATIONLULC"/"$MAPSET" --exec v.proj location=$LOCATION_NAME mapset=$MAPSET input=patch output=patch$RESOLUTION'm'
$grassCMD "$LOCATIONLULC"/"$MAPSET" --exec v.to.rast input=patch$RESOLUTION'm' output=patch$RESOLUTION'm' use=attr attribute_column=value
#
$grassCMD "$LOCATIONLULC"/"$MAPSET" --exec g.region zoom=patch$RESOLUTION'm'
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac.R | $grassCMD "$LOCATIONLULC"/"$MAPSET" --exec R --slave --args patch$RESOLUTION'm' lulcRAW "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv'
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac_write2GIS.R | $grassCMD "$LOCATION"/"$MAPSET" --exec R --slave --args patch "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv' 'https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/lulc_30m_NLCD_remote_forest_catchment.csv'


<div class="alert alert-block alert-info">
(Option 2) NLCD LULC classification for urbanized landscape
</div>

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec r.in.gdal -e --overwrite input="$PROJDIR"/raw_data/'LULC1m.tif' output=lulcRAW location=lulcRAW
LOCATIONLULC="$GISDBASE"/lulcRAW
$grassCMD "$LOCATION"/"$MAPSET" --exec r.to.vect input=patch output=patch type=area
$grassCMD "$LOCATIONLULC"/"$MAPSET" --exec v.proj location=$LOCATION_NAME mapset=$MAPSET input=patch output=patch$RESOLUTION'm'
$grassCMD "$LOCATIONLULC"/"$MAPSET" --exec v.to.rast input=patch$RESOLUTION'm' output=patch$RESOLUTION'm' use=attr attribute_column=value
#
$grassCMD "$LOCATIONLULC"/"$MAPSET" --exec g.region zoom=patch$RESOLUTION'm'
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac.R | $grassCMD "$LOCATIONLULC"/"$MAPSET" --exec R --slave --args patch$RESOLUTION'm' lulcRAW "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv'
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac_write2GIS.R | $grassCMD "$LOCATION"/"$MAPSET" --exec R --slave --args patch "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv' 'https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/lulc_30m_NLCD_urban_catchment.csv'


<div class="alert alert-block alert-info">
(Option 3) Baltimore LULC classification
</div>

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec r.in.gdal -e --overwrite input="$PROJDIR"/raw_data/'LULC1m.tif' output=lulcRAW location=lulcRAW
LOCATIONLULC="$GISDBASE"/lulcRAW
$grassCMD "$LOCATION"/"$MAPSET" --exec r.to.vect input=patch output=patch type=area
$grassCMD "$LOCATIONLULC"/"$MAPSET" --exec v.proj location=$LOCATION_NAME mapset=$MAPSET input=patch output=patch$RESOLUTION'm'
$grassCMD "$LOCATIONLULC"/"$MAPSET" --exec v.to.rast input=patch$RESOLUTION'm' output=patch$RESOLUTION'm' use=attr attribute_column=value
#
$grassCMD "$LOCATIONLULC"/"$MAPSET" --exec g.region zoom=patch$RESOLUTION'm'
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac.R | $grassCMD "$LOCATIONLULC"/"$MAPSET" --exec R --slave --args patch$RESOLUTION'm' lulcRAW "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv'
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac_write2GIS.R | $grassCMD "$LOCATION"/"$MAPSET" --exec R --slave --args patch "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv' 'https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/lulc_1m_Chesapeake_Conservancy.csv'


<div class="alert alert-block alert-info">
(Option 4) Virginia LULC classification
</div>

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec r.in.gdal -e --overwrite input="$PROJDIR"/raw_data/'LULC1m.tif' output=lulcRAW location=lulcRAW
LOCATIONLULC="$GISDBASE"/lulcRAW
$grassCMD "$LOCATION"/"$MAPSET" --exec r.to.vect input=patch output=patch type=area
$grassCMD "$LOCATIONLULC"/"$MAPSET" --exec v.proj location=$LOCATION_NAME mapset=$MAPSET input=patch output=patch$RESOLUTION'm'
$grassCMD "$LOCATIONLULC"/"$MAPSET" --exec v.to.rast input=patch$RESOLUTION'm' output=patch$RESOLUTION'm' use=attr attribute_column=value
#
$grassCMD "$LOCATIONLULC"/"$MAPSET" --exec g.region zoom=patch$RESOLUTION'm'
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac.R | $grassCMD "$LOCATIONLULC"/"$MAPSET" --exec R --slave --args patch$RESOLUTION'm' lulcRAW "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv'
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac_write2GIS.R | $grassCMD "$LOCATION"/"$MAPSET" --exec R --slave --args patch "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv' 'https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/lulc_1m_va.csv'


<div class="alert alert-block alert-info">
(Option 5) For other customized LULC data, 
this block of code below is to create an csv table for users to fill in the composition information for each LULC class
</div>

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec r.in.gdal -e --overwrite input="$PROJDIR"/raw_data/'LULC1m.tif' output=lulcRAW location=lulcRAW
LOCATIONLULC="$GISDBASE"/lulcRAW
$grassCMD "$LOCATION"/"$MAPSET" --exec r.to.vect input=patch output=patch type=area
$grassCMD "$LOCATIONLULC"/"$MAPSET" --exec v.proj location=$LOCATION_NAME mapset=$MAPSET input=patch output=patch$RESOLUTION'm'
$grassCMD "$LOCATIONLULC"/"$MAPSET" --exec v.to.rast input=patch$RESOLUTION'm' output=patch$RESOLUTION'm' use=attr attribute_column=value
#
$grassCMD "$LOCATIONLULC"/"$MAPSET" --exec g.region zoom=patch$RESOLUTION'm'
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac.R | $grassCMD "$LOCATIONLULC"/"$MAPSET" --exec R --slave --args patch$RESOLUTION'm' lulcRAW "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv'
curl -s "$GITHUBLIBRARIES"/LULC_codeinformation.R | R --slave --args "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv' "$PROJDIR"/"$RHESSysNAME"/lulc_codeinformation.csv


<div class="alert alert-block alert-info">
Fill the csv table $PROJDIR / $RHESSysNAME/lulc_codeinformation.csv
</div>

In [ ]:
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac_write2GIS.R | $grassCMD "$LOCATION"/"$MAPSET" --exec R --slave --args patch "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv' "$PROJDIR"/"$RHESSysNAME"/lulc_codeinformation.csv

### 1.10 **Define forest vegetation**

<div class="alert alert-block alert-info">
The forest/shrub/crop/grass LULC composition calculated above is a defined area for forest/shrub/crop/grass, and we have to define what vegetations should be growing in the area. Current script can support up to 15 vegetation types per patch.
</div>

<div class="alert alert-block alert-info">
(Option 1)
    
For example, we model an oak-dominated canopy in a forested catchment. <br>
We determine the locations of oak canopy (the first line) <br>
We simply model the full canopy as oak and maple (the second line); "FFrac" indicates the oak composition in the full canopy. In this example, FFrac = 0.8, i.e., we treat the full canopy as 80% oak and 20% maple.<br>
We set the LAI for the oak canopy (the third line). In this example, LAI is a constant 5.5. One can substitude the 5.5 value by an imported remotely sensed LAI raster. 
</div>

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="oak_canopy = if(forestFrac>0,102,null())" # - the first line
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="oak_canopy_FFrac = if(forestFrac>0,0.8,null())" # - the second line
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="oak_canopy_LAI = if(forestFrac>0,5.5,null())" # - the third line

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="maple_canopy = if(forestFrac>0,111,null())"
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="maple_canopy_FFrac = if(forestFrac>0,0.2,null())"
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="maple_canopy_LAI = if(forestFrac>0,5.5,null())"

<div class="alert alert-block alert-info">
Similarly, we can model the lawn/pasture the same way.
</div>

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="grass1StratumID = if(lawnFrac>0,3,null())"
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="grass1FFrac = if(lawnFrac>0,1.0,null())"
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="grass1LAI = if(lawnFrac>0,1.5,null())"

<div class="alert alert-block alert-info">
(Option 2) Assign canopy type from NLCD dataset (assume the model patch size is 30 m or less):
</div>

In [ ]:
# assume that the NLCD dataset has been imported into lulcRAW from the previous step above.
# $grassCMD "$LOCATION"/"$MAPSET" --exec r.in.gdal -e --overwrite input="$PROJDIR"/raw_data/'NLCD.tif' output=lulcRAW location=lulcRAW
# LOCATIONLULC="$GISDBASE"/lulcRAW
# then we reproject the NLCD onto the modeling GIS dataset
$grassCMD "$LOCATION"/"$MAPSET" --exec r.proj location="$LOCATIONLULC" mapset="PERMANENT" input="lulcRAW" output="NLCD"
#
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="deciduous = if(NLCD==41||NLCD==43||NLCD==90||NLCD==95,0,2,null())" 
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="deciduous_FFrac = if(NLCD==41||NLCD==43||NLCD==90||NLCD==95,0.8,null())" 
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="deciduous_LAI = if(NLCD==41||NLCD==43||NLCD==90||NLCD==95,4.0,null())"
#
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="evergreen = if(NLCD==42||NLCD==52,1,null())" 
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="evergreen_FFrac = if(NLCD==42||NLCD==52,0.8,null())" 
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="evergreen_LAI = if(NLCD==42||NLCD==52,4.0,null())"
#
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="grass1StratumID = if(NLCD==71||NLCD==72||NLCD==81||NLCD==82,3,null())"
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="grass1FFrac = if(NLCD==71||NLCD==72||NLCD==81||NLCD==82,1.0,null())"
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="grass1LAI = if(NLCD==71||NLCD==72||NLCD==81||NLCD==82,1.5,null())"
#

### 1.11 **Import road network (shapefile; if applied)**

<div class="alert alert-block alert-info">
This is optional to import a vector-based road network.
</div>

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec v.in.ogr --overwrite input="$PROJDIR"/raw_data/'Roads_GDT_MSA.shp' output=roads location=roadRAW
LOCATIONROAD="$GISDBASE"/roadRAW
$grassCMD "$LOCATION"/"$MAPSET" --exec v.proj --overwrite location=roadRAW mapset=$MAPSET input=roads output=roads
$grassCMD "$LOCATION"/"$MAPSET" --exec v.to.rast --overwrite input=roads output=vector_roads use=cat
rm -rf "$LOCATIONROAD"

### 1.12 **Define riparian (if applied)**

<div class="alert alert-block alert-info">
(Option 1) import a determined riparian raster
</div>

In [ ]:
$grassCMD "$LOCATION"/"$MAPSET" --exec r.in.gdal -o --overwrite input="$PROJDIR"/raw_data/riparian.tif output=riparian

<div class="alert alert-block alert-info">
(Option 2) Use HANDS tool to define riparian zone. This is timely process.
</div>

In [ ]:
curl -s "$GITHUBLIBRARIES"/elevation_analysis.R | $grassCMD "$LOCATION"/$MAPSET --exec R --slave --args dem colmap rowmap drain hill strExt
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc expression="riparian_hands = if( handsDEM < 5, 1, null())" --overwrite

### 1.13 **Define surface drainage features for urban catchments (if applied)**

<div class="alert alert-block alert-info">
Current RHESSys EC supports storm drinage along road network, (subsurface) sewer drinage, and surface drainage. 
</div>

In [ ]:
### ... storm drinage along the paved roads
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roadDEM = if(pavedroadFrac>0,dem,null())"
$grassCMD "$LOCATION"/"$MAPSET" --exec r.watershed -s --overwrite elevation=roadDEM drainage=roadDrain
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roadExit = if(roadDrain<0, patch, null())"
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roadExitDEM = if(isnull(roadExit),null(),dem)"
$grassCMD "$LOCATION"/"$MAPSET" --exec r.to.vect --overwrite input=roadExit output=roadExit type=point column=inPatch
#
$grassCMD "$LOCATION"/"$MAPSET" --exec r.buffer --overwrite input=roadExit output=roadExit_buffer distances=120
$grassCMD "$LOCATION"/"$MAPSET" --exec r.buffer --overwrite input=roadDEM output=road_buffer distances=30
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roadExit_neighbor_area = if(roadExit_buffer>1 & isnull(road_buffer),  if(isnull(strExt) & impFrac==0,1, null()), null())"
$grassCMD "$LOCATION"/"$MAPSET" --exec r.clump --overwrite -d input=roadExit_neighbor_area output=roadExit_neighbor_id
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roadExit_neighbor_usdDEM = if(roadExit_buffer>1 & isnull(road_buffer),  if(isnull(strExt) & impFrac==0,usdDEM, null()), null())"
curl -s "$GITHUBLIBRARIES"/roadstormdriangeOutlet.R | $grassCMD "$LOCATION"/$MAPSET --exec R --slave --args basin roadExitDEM roadExit_neighbor_id roadExit_neighbor_usdDEM dem xmap ymap patch roadExit_neighbor_usdDEMmed roadExitOutletPatchID roadExitOutlet 120
$grassCMD "$LOCATION"/"$MAPSET" --exec r.to.vect --overwrite input=roadExitOutlet output=roadExitOutlet type=point column=inPatch

In [ ]:
### ... rooftop surface routing
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roofQ = if(roofFrac>0,1,null())"
$grassCMD "$LOCATION"/"$MAPSET" --exec r.grow --overwrite input=roofQ output=roofQgrow radius=1.51
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roofDEM = if(isnull(roofQgrow),null(),if(isnull(roofQ),dem,dem+6))"
$grassCMD "$LOCATION"/"$MAPSET" --exec r.watershed -s --overwrite elevation=roofDEM drainage=roofDrain_
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roofDrain = if(isnull(roofQ),null(),roofDrain_)"
### ... driveway surface routing
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roaddrivewayDEM = if(pavedroadFrac_va1mFilled>0,if(drivewayFrac_va1mFilled>0, dem, null()),null())"
$grassCMD "$LOCATION"/"$MAPSET" --exec r.watershed -s --overwrite elevation=roaddrivewayDEM drainage=roaddrivewayDrain

In [ ]:
### ... assuming sewer lines go along roads, we simply use the "pavedroadFrac" as the sewer map
# if explicit sewer network data is available, import it to the model GISdataset and cover it to raster.

In [ ]:
### ... additional rapid surface drainage  (other than roof top, driveway, parking, and paved roads), e.g., drainage hole in playground / garden
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="addsurfdrain = if( forestFrac<0.5 && roofBuff>0, 1 , null())"

In [ ]:
### ... soil compactness by construction
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roof = if(roofFrac > 0.5, 1, null())"
$grassCMD "$LOCATION"/"$MAPSET" --exec r.buffer --overwrite input=roof output=roofBuff distances=30
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="naturalLand = if( (forestFrac>=1 || lawnFrac>=1)&& isnull(roofBuff), 1 , null())"
$grassCMD "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="compactedsoil = if( naturalLand>0, null(), basin)"

In [ ]:
### ... well water supply, lawn irrigation, and septic (calculations here require 1-m high resolution elevation and LULC dataset)
#
# first identify rooftops and lawns from the 1-m LULC dataset
# code 7 & 10 are rooftops
$grassCMD "$LOCATIONLULC"/$MAPSET --exec r.mapcalc expression="rooftops = if(lulcRAW==7 || lulcRAW==10,1,null())"
$grassCMD "$LOCATIONLULC"/$MAPSET --exec r.clump -d --overwrite input=rooftops output=rooftopIDs
$grassCMD "$LOCATIONLULC"/$MAPSET --exec r.to.vect input=rooftopIDs output=rooftopIDs type=area
# code 5 is lawn & garden
$grassCMD "$LOCATIONLULC"/$MAPSET --exec r.mapcalc expression="lawnfields = if(lulcRAW==5,1,null())"
$grassCMD "$LOCATIONLULC"/$MAPSET --exec r.clump -d --overwrite input=lawnfields output=lawnfieldIDs
$grassCMD "$LOCATIONLULC"/$MAPSET --exec r.to.vect input=lawnfieldIDs output=lawnfieldIDs type=area
# import back to project gis
$grassCMD "$LOCATION"/$MAPSET --exec v.proj location=lulcRAW mapset=$MAPSET input=rooftopIDs output=rooftopIDs
$grassCMD "$LOCATION"/$MAPSET --exec v.proj location=lulcRAW mapset=$MAPSET input=lawnfieldIDs output=lawnfieldIDs
$grassCMD "$LOCATION"/$MAPSET --exec v.extract --overwrite input=rooftopIDs type=boundary,centroid output=rooftopIDsCentroid
#
# second import rooftops, lawns, and model patch IDs to elevation gis
$grassCMD "$LOCATIONDEM"/$MAPSET --exec v.proj location=lulcRAW mapset=$MAPSET input=rooftopIDs output=rooftopIDs
$grassCMD "$LOCATIONDEM"/$MAPSET --exec v.proj location=lulcRAW mapset=$MAPSET input=lawnfieldIDs output=lawnfieldIDs
$grassCMD "$LOCATIONDEM"/$MAPSET --exec v.proj location=$LOCATION_NAME mapset=$MAPSET input=patch output=patch$RESOLUTION'm'
$grassCMD "$LOCATIONDEM"/$MAPSET --exec v.to.rast input=patch$RESOLUTION'm' output=patch$RESOLUTION'm' use=attr attribute_column=value
$grassCMD "$LOCATIONDEM"/$MAPSET --exec v.to.rast input=rooftopIDs output=rooftopIDs use=attr attribute_column=value
$grassCMD "$LOCATIONDEM"/$MAPSET --exec v.to.rast input=lawnfieldIDs output=lawnfieldIDs use=attr attribute_column=value
$grassCMD "$LOCATIONDEM"/$MAPSET --exec v.db.addcolumn map=rooftopIDs columns="lawnID integer" 
$grassCMD "$LOCATIONDEM"/$MAPSET --exec v.distance from=rooftopIDs from_type=point,line,boundary,area to=lawnfieldIDs upload=to_attr column=lawnID to_column=value
$grassCMD "$LOCATIONDEM"/$MAPSET --exec v.to.rast input=rooftopIDs output=rooftopIDlawnIDs use=attr attribute_column=lawnID
$grassCMD "$LOCATIONDEM"/$MAPSET --exec r.mapcalc expression="xmap = x()"
$grassCMD "$LOCATIONDEM"/$MAPSET --exec r.mapcalc expression="ymap = y()"
# read in R for processing (this outputs the csv tables to document the potential well location in patchID and the potential septic drainage field locations at the possible downslope of the houses)
curl -s "$GITHUBLIBRARIES"/well_septicDrainField_IrrigrationLawn_around_rooftops.R | $grassCMD "$LOCATIONDEM"/$MAPSET --exec R --slave --args patch30m demRAW lawnfieldIDs rooftopIDs rooftopIDlawnIDs xmap ymap "$PROJDIR"/"$RHESSysNAME"/septic_IN_OUT.csv "$PROJDIR"/"$RHESSysNAME"/lawnIrrigration_IN_OUT.csv